# Meta-Prompt

This is an implementation of [meta-prompt](https://noahgoodman.substack.com/p/meta-prompt-a-simple-self-improving?sd=pf#footnote-1-112242455) from [Noah D. Goodman](https://cocolab.stanford.edu/ndg) written using [LangChain](https://github.com/hwchase17/langchain) by [Harrison Chase](https://twitter.com/hwchase17).

In [1]:
from langchain import OpenAI, ConversationChain, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory

In [2]:
def initialize_chain(instructions, memory=None):
    if memory is None:
        memory = ConversationBufferWindowMemory()
        memory.ai_prefix = "Assistant"

    template = f"""
    Instructions: {instructions}
    {{{memory.memory_key}}}
    Human: {{human_input}}
    Assistant:"""

    prompt = PromptTemplate(
        input_variables=["history", "human_input"], 
        template=template
    )

    chain = LLMChain(
        llm=OpenAI(temperature=0), 
        prompt=prompt, 
        verbose=True, 
        memory=ConversationBufferWindowMemory(),
    )
    return chain
    
def initialize_meta_chain():
    meta_template="""
    Assistant has just had the below interactions with a User. Assistant followed their "Instructions" closely. Your job is to critique the Assistant's performance and then revise the Instructions so that Assistant would quickly and correctly respond in the future.

    ####

    {chat_history}

    ####

    Please reflect on these interactions.

    You should first critique Assistant's performance. What could Assistant have done better? What should the Assistant remember about this user? Are there things this user always wants? Indicate this with "Critique: ...".

    You should next revise the Instructions so that Assistant would quickly and correctly respond in the future. Assistant's goal is to satisfy the user in as few interactions as possible. Assistant will only see the new Instructions, not the interaction history, so anything important must be summarized in the Instructions. Don't forget any important details in the current Instructions! Indicate the new Instructions by "Instructions: ...".
    """

    meta_prompt = PromptTemplate(
        input_variables=["chat_history"], 
        template=meta_template
    )

    meta_chain = LLMChain(
        llm=OpenAI(temperature=0), 
        prompt=meta_prompt, 
        verbose=True, 
    )
    return meta_chain
    
def get_chat_history(chain_memory):
    memory_key = chain_memory.memory_key
    chat_history = chain_memory.load_memory_variables(memory_key)[memory_key]
    return chat_history

def get_new_instructions(meta_output):
    delimiter = 'Instructions: '
    new_instructions = meta_output[meta_output.find('Instructions: ')+len(delimiter):]
    return new_instructions

In [25]:
def main(task, max_iters=5, max_meta_iters=5):
    failed_phrase = 'task failed'
    success_phrase = 'task succeeded'
    key_phrases = [success_phrase, failed_phrase]
    
    instructions = 'None'
    for i in range(max_meta_iters):
        print(f'[Episode {i+1}/{max_meta_iters}]')
        chain = initialize_chain(instructions, memory=None)
        output = chain.predict(human_input=task)
        for j in range(max_iters):
            print(f'(Step {j+1}/{max_iters})')
            print(f'Assistant: {output}')
            print(f'Human: ')
            human_input = input()
            if any(phrase in human_input.lower() for phrase in key_phrases):
                break
            output = chain.predict(human_input=human_input)
        if success_phrase in human_input.lower():
            print(f'You succeeded! Thanks for playing!')
            return
        meta_chain = initialize_meta_chain()
        meta_output = meta_chain.predict(chat_history=get_chat_history(chain.memory))
        print(f'Feedback: {meta_output}')
        instructions = get_new_instructions(meta_output)
        print(f'New Instructions: {instructions}')
        print('\n'+'#'*80+'\n')
    print(f'You failed! Thanks for playing!')

In [26]:
task = "Guess what I'm thinking about. You can ask yes/no questions."
main(task)

[Episode 1/5]


> Entering new LLMChain chain...
Prompt after formatting:

    Instructions: None
    
    Human: Guess what I'm thinking about. You can ask yes/no questions.
    Assistant:

> Finished chain.
(Step 1/5)
Assistant:  Is it an animal?
Human: 
no


> Entering new LLMChain chain...
Prompt after formatting:

    Instructions: None
    Human: Guess what I'm thinking about. You can ask yes/no questions.
AI:  Is it an animal?
    Human: no
    Assistant:

> Finished chain.
(Step 2/5)
Assistant:  Is it a place?
Human: 
no


> Entering new LLMChain chain...
Prompt after formatting:

    Instructions: None
    Human: Guess what I'm thinking about. You can ask yes/no questions.
AI:  Is it an animal?
Human: no
AI:  Is it a place?
    Human: no
    Assistant:

> Finished chain.
(Step 3/5)
Assistant:  Is it an object?
Human: 
yes


> Entering new LLMChain chain...
Prompt after formatting:

    Instructions: None
    Human: Guess what I'm thinking about. You can ask yes/no questions.
A


> Finished chain.
Feedback: 
Critique: The Assistant should have asked more specific questions to narrow down the possibilities. It should have asked questions that would help it understand the user's needs better.

Instructions: Ask questions that will help the Assistant understand the user's needs better. Try to narrow down the possibilities by asking more specific questions.
New Instructions: Ask questions that will help the Assistant understand the user's needs better. Try to narrow down the possibilities by asking more specific questions.

################################################################################

[Episode 3/5]


> Entering new LLMChain chain...
Prompt after formatting:

    Instructions: Ask questions that will help the Assistant understand the user's needs better. Try to narrow down the possibilities by asking more specific questions.
    
    Human: Guess what I'm thinking about. You can ask yes/no questions.
    Assistant:

> Finished chain.
(Step 1/5)



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    Assistant has just had the below interactions with a User. Assistant followed their "Instructions" closely. Your job is to critique the Assistant's performance and then revise the Instructions so that Assistant would quickly and correctly respond in the future.

    ####

    Human: no
AI:  Is it an object?
Human: yes
AI:  Is it smaller than a shoe?
Human: yes
AI:  Is it something you can touch?
Human: yes
AI:  Is it a coin?
Human: no
AI:  Is it a rock?

    ####

    Please reflect on these interactions.

    You should first critique Assistant's performance. What could Assistant have done better? What should the Assistant remember about this user? Are there things this user always wants? Indicate this with "Critique: ...".

    You should next revise the Instructions so that Assistant would quickly and correctly respond in the future. Assistant's goal is to satisfy the user in as few interactions as p